In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import folium
import json
import dateutil.parser
import datetime as dt
from datetime import datetime
from scipy.interpolate import interp1d
from functools import reduce
import pycountry

from utils import preprocessing

In [2]:
entities = r'./panama_csv/Entities.csv'
entities = pd.read_csv(entities,index_col='name', header=0, low_memory=False)
entities=entities.rename(columns = {'countries':'Country','jurisdiction':'abbr_jurisd','jurisdiction_description':'jurisdiction'})

In [3]:
all_countries = entities.groupby('Country').count().index

In [4]:
countries_frame = preprocessing.process_countries_with_code(entities, 
                                                            all_countries, 
                                                            analisys_on='Country', 
                                                            from_year=1900, 
                                                            to_year=2017,
                                                            feature='Active offshores'
                                                           )

Country done:Albania
Country done:American Samoa
Country done:Andorra
Country done:Andorra;Not identified
Country done:Angola
Country done:Anguilla
Country done:Antigua and Barbuda
Country done:Antigua and Barbuda;British Virgin Islands
Country done:Antigua and Barbuda;Not identified
Country done:Argentina
Country done:Aruba
Country done:Australia
Country done:Australia;Belize
Country done:Australia;British Virgin Islands
Country done:Australia;Not identified
Country done:Austria
Country done:Austria;British Virgin Islands
Country done:Austria;Not identified
Country done:Azerbaijan
Country done:Bahamas
Country done:Bahamas;Not identified
Country done:Bahamas;Panama
Country done:Bahrain
Country done:Bangladesh
Country done:Barbados
Country done:Belarus
Country done:Belarus;British Virgin Islands
Country done:Belarus;Not identified
Country done:Belgium
Country done:Belgium;British Virgin Islands
Country done:Belgium;Not identified
Country done:Belize
Country done:Belize;British Virgin Is

/Users/Pucci/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:1457: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


Country done:Cayman Islands;Taiwan
Country done:Cayman Islands;Taiwan;British Virgin Islands
Country done:Central African Republic
Country done:Chad
Country done:Chile
Country done:China
Country done:China;British Virgin Islands
Country done:China;Not identified
Country done:Colombia
Country done:Cook Islands
Country done:Cook Islands;Cayman Islands
Country done:Cook Islands;Hong Kong
Country done:Cook Islands;Indonesia
Country done:Cook Islands;Not identified
Country done:Cook Islands;Singapore
Country done:Cook Islands;Taiwan
Country done:Cook Islands;United States
Country done:Costa Rica
Country done:Costa Rica;Not identified
Country done:Croatia
Country done:Croatia;British Virgin Islands
Country done:Croatia;Not identified
Country done:Cuba
Country done:Curaçao
Country done:Curaçao;British Virgin Islands
Country done:Curaçao;Netherlands;Not identified
Country done:Curaçao;Not identified
Country done:Cyprus
Country done:Cyprus;British Virgin Islands
Country done:Cyprus;Not identifi

Country done:Romania;Not identified
Country done:Russia
Country done:Russia;British Virgin Islands
Country done:Russia;Not identified
Country done:Russia;Saint Kitts and Nevis
Country done:Saint Kitts and Nevis
Country done:Saint Kitts and Nevis;British Virgin Islands
Country done:Saint Kitts and Nevis;Canada
Country done:Saint Kitts and Nevis;Latvia
Country done:Saint Kitts and Nevis;Not identified
Country done:Saint Kitts and Nevis;Russia
Country done:Saint Kitts and Nevis;Sweden
Country done:Saint Kitts and Nevis;United Arab Emirates
Country done:Saint Kitts and Nevis;United Kingdom
Country done:Saint Kitts and Nevis;United States
Country done:Saint Lucia
Country done:Saint Vincent and the Grenadines
Country done:Samoa
Country done:Samoa;Cayman Islands
Country done:Samoa;Cook Islands
Country done:Samoa;Hong Kong
Country done:Samoa;Malaysia
Country done:Samoa;Not identified
Country done:Samoa;Singapore
Country done:Samoa;Taiwan
Country done:Samoa;Thailand
Country done:Saudi Arabia
Co

In [5]:
# countries_frame.to_csv('../../data/csv/cash_flows_actives.csv', index=False)

In [6]:
# countries_frame_wname = countries_frame.groupby('Country').sum().reset_index()
# countries_frame_name = countries_frame.groupby(['Country','Country_name']).sum().reset_index()